In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("WELFake_Dataset.csv.zip", index_col=0)

In [3]:
data.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
df = data.copy()

In [5]:
df.shape

(72134, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [7]:
df.isnull().sum()

title    558
text      39
label      0
dtype: int64

In [8]:
df.dropna(subset=['title', 'text'], inplace=True)

In [9]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [10]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [11]:
df.shape

(71537, 3)

In [12]:
df.nunique()

title    62308
text     62200
label        2
dtype: int64

In [13]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords as nltk_stopwords

In [14]:
# Ensure stopwords are available; handle offline environments
try:
    nltk.data.find('corpora/stopwords')
    _have_stopwords = True
except LookupError:
    _have_stopwords = nltk.download('stopwords')


In [15]:
# Build a stopword set with robust fallbacks
try:
    STOPWORDS = set(nltk_stopwords.words('english'))
except LookupError:
    try:
        from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
        STOPWORDS = set(ENGLISH_STOP_WORDS)
    except Exception:
        STOPWORDS = {
            'the','a','an','and','or','if','is','it','this','that','to','for','of','on','in','with','as','by','at','from',
            'be','are','was','were','been','has','have','had','do','does','did','but','not','no','so','than','too','very'
        }


In [16]:
# Ensure wordnet is available; handle offline environments
try:
    nltk.data.find('corpora/wordnet')
    _have_wordnet = True
except LookupError:
    _have_wordnet = nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nagad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# Text cleaning helper with safe fallbacks
try:
    lemmatizer = WordNetLemmatizer()
    _use_lemma = True
except Exception:
    lemmatizer = None
    _use_lemma = False

stemmer = PorterStemmer()


def _clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ''
    txt = text.lower()
    txt = BeautifulSoup(txt, 'html.parser').get_text()
    txt = re.sub(r'[^a-z0-9 ]+', ' ', txt)
    tokens = txt.split()

    if _use_lemma:
        try:
            tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in STOPWORDS]
        except LookupError:
            tokens = [stemmer.stem(w) for w in tokens if w not in STOPWORDS]
    else:
        tokens = [stemmer.stem(w) for w in tokens if w not in STOPWORDS]

    return ' '.join(tokens)


In [18]:
# Apply cleaner to the text column and store result
# Avoid passing a DataFrame to the string-cleaning function
df['clean_text'] = df['text'].astype(str).map(_clean_text)

df[['text', 'clean_text']].head()

C:\Users\nagad\AppData\Local\Temp\ipykernel_19816\747544523.py:16: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  txt = BeautifulSoup(txt, 'html.parser').get_text()


,text,clean_text
0,No comment is expected from Barack Obama Membe...,comment expected barack obama member fyf911 fu...
2,"Now, most of the demonstrators gathered last ...",demonstrator gathered last night exercising co...
3,A dozen politically active pastors came here f...,dozen politically active pastor came private d...
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",r 28 sarmat missile dubbed satan 2 replace s 1...
5,All we can say on this one is it s about time ...,say one time someone sued southern poverty law...


In [19]:
df

,title,text,label,clean_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,comment expected barack obama member fyf911 fu...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,demonstrator gathered last night exercising co...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,dozen politically active pastor came private d...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,r 28 sarmat missile dubbed satan 2 replace s 1...
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,say one time someone sued southern poverty law...
...,...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hacker believed working rus...
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,know fantasyland republican never questioned c...
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrant refuse leave train refugee camp hungar...
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump combative sty...
